<a href="https://colab.research.google.com/github/SarinaMashreghi/monkey_pox_application/blob/main/monkeypox_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import os
import time
import matplotlib.pyplot as plt
import cv2
import seaborn as sns
import shutil
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation,Dropout,Conv2D, MaxPooling2D,BatchNormalization
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model

In [ ]:
sdir=r'../content/drive/MyDrive/Original Images'
filepaths=[]
labels=[]
classlist=os.listdir(sdir)   
for klass in classlist:
    classpath=os.path.join(sdir, klass)
    flist=os.listdir(classpath)
    for f in flist:
        fpath=os.path.join(classpath,f)
        filepaths.append(fpath)
        labels.append(klass)

print(labels)

['Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monkey Pox', 'Monk

In [ ]:
Fseries=pd.Series(filepaths, name='filepaths')
Lseries=pd.Series(labels, name='labels')        
df=pd.concat([Fseries, Lseries], axis=1)
train_df, dummy_df=train_test_split(df, train_size=.75, shuffle=True, random_state=123, stratify=df['labels'])
valid_df, test_df=train_test_split(dummy_df, train_size=.5, shuffle=True, random_state=123, stratify=dummy_df['labels'])
print('train_df lenght: ', len(train_df), '  test_df length: ', len(test_df), '  valid_df length: ', len(valid_df))

train_df lenght:  171   test_df length:  29   valid_df length:  28


##Augmenting images

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def balance(df, n, working_dir, img_size):
    df=df.copy()
    print('Initial length: ', len(df))
    aug_dir=os.path.join(working_dir, 'aug') # directory to store augmented images
    if os.path.isdir(aug_dir):# start with an empty directory
        shutil.rmtree(aug_dir)
    os.mkdir(aug_dir)        
    for label in df['labels'].unique():    
        dir_path=os.path.join(aug_dir,label)    
        os.mkdir(dir_path) # make class directories within aug directory
    # create and store the augmented images  
    total=0
    gen=ImageDataGenerator(horizontal_flip=True,  rotation_range=20, width_shift_range=.2,
                                  height_shift_range=.2, zoom_range=.2)
    groups=df.groupby('labels') # group by class
    for label in df['labels'].unique():  # for every class               
        group=groups.get_group(label)  # a dataframe holding only rows with the specified label 
        sample_count=len(group)   # determine how many samples there are in this class        
        if sample_count< n: # if the class has less than target number of images
            aug_img_count=0
            delta=n - sample_count  # number of augmented images to create
            target_dir=os.path.join(aug_dir, label)  # define where to write the images
            msg='{0:40s} for class {1:^30s} creating {2:^5s} augmented images'.format(' ', label, str(delta))
            print(msg, '\r', end='') # prints over on the same line
            aug_gen=gen.flow_from_dataframe( group,  x_col='filepaths', y_col=None, target_size=img_size,
                                            class_mode=None, batch_size=1, shuffle=False, 
                                            save_to_dir=target_dir, save_prefix='aug-', color_mode='rgb',
                                            save_format='jpg')
            while aug_img_count<delta:
                images=next(aug_gen)            
                aug_img_count += len(images)
            total +=aug_img_count
    print('Total Augmented images: ', total)
    # create aug_df and merge with train_df to create composite training set ndf
    aug_fpaths=[]
    aug_labels=[]
    classlist=os.listdir(aug_dir)
    for klass in classlist:
        classpath=os.path.join(aug_dir, klass)     
        flist=os.listdir(classpath)    
        for f in flist:        
            fpath=os.path.join(classpath,f)         
            aug_fpaths.append(fpath)
            aug_labels.append(klass)
    Fseries=pd.Series(aug_fpaths, name='filepaths')
    Lseries=pd.Series(aug_labels, name='labels')
    aug_df=pd.concat([Fseries, Lseries], axis=1)         
    df=pd.concat([df,aug_df], axis=0).reset_index(drop=True)
    print('Length of augmented dataframe: ', len(df))
    return df 

n=200 # number of samples in each class
working_dir=r'./' # directory to store augmented images
img_size=(224,224) # size of augmented images
train_df=balance(train_df, n, working_dir, img_size)    

Initial length:  171
Found 76 validated image filenames.
Found 95 validated image filenames.
Total Augmented images:  229
Length of augmented dataframe:  400


In [ ]:
batch_size = 20 
trgn = ImageDataGenerator(horizontal_flip=True, rotation_range=20, width_shift_range=.2,
                                  height_shift_range=.2, zoom_range=.2 )
t_and_v_gen=ImageDataGenerator()
msg='{0:70s} for train generator'.format(' ')
print(msg, '\r', end='') 
train_gen=trgn . flow_from_dataframe(train_df, x_col='filepaths', y_col='labels', target_size=img_size,
                                   class_mode='categorical', color_mode='rgb', shuffle=True, batch_size=batch_size)
msg='{0:70s} for valid generator'.format(' ')
print(msg, '\r', end='') 
valid_gen=t_and_v_gen.flow_from_dataframe(valid_df, x_col='filepaths', y_col='labels', target_size=img_size,
                                   class_mode='categorical', color_mode='rgb', shuffle=False, batch_size=batch_size)
length=len(test_df)
test_batch_size=sorted([int(length/n) for n in range(1,length+1) if length % n ==0 and length/n<=80],reverse=True)[0]  
test_steps=int(length/test_batch_size)
msg='{0:70s} for test generator'.format(' ')
print(msg, '\r', end='') 
test_gen=t_and_v_gen.flow_from_dataframe(test_df, x_col='filepaths', y_col='labels', target_size=img_size,
                                   class_mode='categorical', color_mode='rgb', shuffle=False, batch_size=test_batch_size)

classes=list(train_gen.class_indices.keys())
class_indices=list(train_gen.class_indices.values())
class_count=len(classes)
labels=test_gen.labels
print ( 'test batch size: ' ,test_batch_size, '  test steps: ', test_steps, ' number of classes : ', class_count)

Found 400 validated image filenames belonging to 2 classes.
Found 28 validated image filenames belonging to 2 classes.
Found 29 validated image filenames belonging to 2 classes.
test batch size:  29   test steps:  1  number of classes :  2


##EfficientNetB7

In [ ]:
img_shape = (img_size[0], img_size[1], 3)

base_model = tf.keras.applications.efficientnet.EfficientNetB7(include_top=False, weights="imagenet",input_shape=img_shape, pooling='max')

base_model.trainable = True #not freezing 

x = base_model.output
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 )(x)
x = Dense(512, kernel_regularizer = regularizers.l2(l = 0.016),activity_regularizer=regularizers.l1(0.006),
                bias_regularizer=regularizers.l1(0.006) ,activation='leaky_relu')(x)

x=Dropout(rate=0.2)(x)       
output=Dense(class_count, activation='softmax')(x)
model=Model(inputs=base_model.input, outputs=output)
lr=0.001 
model.compile(Adamax(learning_rate=lr), loss='binary_crossentropy', metrics=['accuracy']) 

258076736/258076736 [==============================] - 12s 0us/step


In [ ]:
!pip install livelossplot
from livelossplot import PlotLossesKeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 22.5 MB/s 


In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping

epochs = 100
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=0.00001)
early_stopping = EarlyStopping(monitor="val_accuracy", baseline=0.9, restore_best_weights=True, mode="max")

In [ ]:
# callbacks=[reduce_lr, early_stopping]
callbacks=[reduce_lr]
history=model.fit(x=train_gen,  epochs=epochs, verbose=1, callbacks=callbacks, validation_data=valid_gen)

Epoch 1/100
20/20 [==============================] - 89s 2s/step - loss: 14.8353 - accuracy: 0.7200 - val_loss: 15.2599 - val_accuracy: 0.6786 - lr: 0.0010
Epoch 2/100
20/20 [==============================] - 25s 1s/step - loss: 11.7704 - accuracy: 0.8800 - val_loss: 11.3265 - val_accuracy: 0.8214 - lr: 0.0010
Epoch 3/100
20/20 [==============================] - 24s 1s/step - loss: 9.4671 - accuracy: 0.9050 - val_loss: 9.2745 - val_accuracy: 0.6786 - lr: 0.0010
Epoch 4/100
20/20 [==============================] - 24s 1s/step - loss: 7.6869 - accuracy: 0.9475 - val_loss: 7.7362 - val_accuracy: 0.7500 - lr: 0.0010
Epoch 5/100
20/20 [==============================] - 24s 1s/step - loss: 6.4470 - accuracy: 0.9525 - val_loss: 6.5825 - val_accuracy: 0.7857 - lr: 0.0010
Epoch 6/100
20/20 [==============================] - 24s 1s/step - loss: 5.5403 - accuracy: 0.9500 - val_loss: 5.5719 - val_accuracy: 0.8214 - lr: 0.0010
Epoch 7/100
20/20 [==============================] - 24s 1s/step - loss:

In [ ]:
from traitlets.traitlets import enum
from sklearn.metrics import confusion_matrix

preds = model.predict(test_gen)

y_pred = []

for i, p in enumerate(preds):
  p = np.argmax(p)
  y_pred.append(p)

confusion_matrix(test_gen.labels, y_pred)

1/1 [==============================] - 6s 6s/step


array([[11,  2],
       [ 0, 16]])

###Saving the model

In [ ]:
model.save("/content/drive/MyDrive/ML/efficientnNetB7_model.h5")

In [ ]:
model_saved = tf.keras.models.load_model("/content/drive/MyDrive/ML/efficientnNetB7_model.h5")
model_saved.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 224, 224, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 tf.math.truediv (TFOpLambda)   (None, 224, 224, 3)  0           ['normalization[0][0]']      

In [ ]:
from tensorflow import lite
import tensorflow as tf
converter = lite.TFLiteConverter.from_keras_model(model_saved)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tfmodel = converter.convert()
open("/content/drive/MyDrive/ML/efficientnNetB7_model_optimized.tflite", "wb").write(tfmodel)

68660704

##Model 2 - B7


In [ ]:
base_model2 = tf.keras.applications.efficientnet.EfficientNetB7(include_top=False, weights="imagenet",input_shape=img_shape, pooling='max')

base_model2.trainable = False

x = base_model2.output
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 )(x)
x = Dense(512, kernel_regularizer = regularizers.l2(l = 0.016),activity_regularizer=regularizers.l1(0.006),
                bias_regularizer=regularizers.l1(0.006) ,activation='leaky_relu')(x)

x = Dropout(rate=0.2)(x)       
output = Dense(class_count, activation='softmax')(x)
model2 = Model(inputs=base_model2.input, outputs=output)
lr = 0.001 
model2.compile(Adamax(learning_rate=lr), loss='binary_crossentropy', metrics=['accuracy']) 

In [ ]:
epochs = 30
callbacks=[reduce_lr]
history2 = model2.fit(x=train_gen,  epochs=epochs, verbose=1, callbacks=callbacks,  validation_data=valid_gen,
               validation_steps=None,  shuffle=False,  initial_epoch=0)

Epoch 1/30
20/20 [==============================] - 7s 324ms/step - loss: 0.6938 - accuracy: 0.9450 - val_loss: 0.9651 - val_accuracy: 0.7143 - lr: 5.0000e-04
Epoch 2/30
20/20 [==============================] - 7s 323ms/step - loss: 0.6518 - accuracy: 0.9525 - val_loss: 0.9783 - val_accuracy: 0.6786 - lr: 5.0000e-04
Epoch 3/30
20/20 [==============================] - 8s 385ms/step - loss: 0.6525 - accuracy: 0.9475 - val_loss: 0.8477 - val_accuracy: 0.7500 - lr: 5.0000e-04
Epoch 4/30
20/20 [==============================] - 7s 327ms/step - loss: 0.6411 - accuracy: 0.9400 - val_loss: 0.8194 - val_accuracy: 0.7857 - lr: 5.0000e-04
Epoch 5/30
20/20 [==============================] - 8s 380ms/step - loss: 0.6306 - accuracy: 0.9300 - val_loss: 0.8364 - val_accuracy: 0.7857 - lr: 5.0000e-04
Epoch 6/30
20/20 [==============================] - 7s 323ms/step - loss: 0.5914 - accuracy: 0.9550 - val_loss: 0.8086 - val_accuracy: 0.8214 - lr: 5.0000e-04
Epoch 7/30
20/20 [============================

##EfficientNetB3

In [ ]:
base_model3 = tf.keras.applications.efficientnet.EfficientNetB3(include_top=False, weights="imagenet",input_shape=img_shape, pooling='max')

base_model3.trainable = True

x = base_model3.output
x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 )(x)
x = Dense(512, kernel_regularizer = regularizers.l2(l = 0.016),activity_regularizer=regularizers.l1(0.006),
                bias_regularizer=regularizers.l1(0.006) ,activation='leaky_relu')(x)

x = Dropout(rate=0.2)(x)       
output = Dense(class_count, activation='softmax')(x)
model3 = Model(inputs=base_model3.input, outputs=output)
lr = 0.001
model3.compile(Adamax(learning_rate=lr), loss='binary_crossentropy', metrics=['accuracy']) 

43941136/43941136 [==============================] - 0s 0us/step


In [ ]:
epochs = 150
callbacks=[reduce_lr]
history3 = model3.fit(x=train_gen,  epochs=epochs, verbose=1, callbacks=callbacks,  validation_data=valid_gen,
               validation_steps=None,  shuffle=False,  initial_epoch=0)

Epoch 1/150
20/20 [==============================] - 7s 328ms/step - loss: 0.4066 - accuracy: 0.9850 - val_loss: 0.6596 - val_accuracy: 0.7857 - lr: 1.0000e-05
Epoch 2/150
20/20 [==============================] - 7s 321ms/step - loss: 0.4531 - accuracy: 0.9475 - val_loss: 0.6599 - val_accuracy: 0.7500 - lr: 1.0000e-05
Epoch 3/150
20/20 [==============================] - 7s 321ms/step - loss: 0.4212 - accuracy: 0.9725 - val_loss: 0.6612 - val_accuracy: 0.7500 - lr: 1.0000e-05
Epoch 4/150
20/20 [==============================] - 7s 324ms/step - loss: 0.4454 - accuracy: 0.9550 - val_loss: 0.6601 - val_accuracy: 0.7500 - lr: 1.0000e-05
Epoch 5/150
20/20 [==============================] - 7s 321ms/step - loss: 0.4414 - accuracy: 0.9625 - val_loss: 0.6610 - val_accuracy: 0.7500 - lr: 1.0000e-05
Epoch 6/150
20/20 [==============================] - 8s 375ms/step - loss: 0.4270 - accuracy: 0.9700 - val_loss: 0.6596 - val_accuracy: 0.7500 - lr: 1.0000e-05
Epoch 7/150
20/20 [=====================

###Saving the model

In [ ]:
from tensorflow import lite

converter = lite.TFLiteConverter.from_keras_model(model3)

tfmodel = converter.convert()
open("/content/drive/MyDrive/ML/efficientnNetB3_model.tflite", "wb").write(tfmodel)

NameError: ignored

##ResNet

In [ ]:
img_shape = (img_size[0], img_size[1], 3) 
base_model4 = tf.keras.applications.resnet50.ResNet50(include_top=False, weights="imagenet",input_shape=img_shape, pooling='max')

base_model4.trainable = True

x = base_model4.output
# x = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 )(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = Dense(512, kernel_regularizer = regularizers.l2(l = 0.016),activity_regularizer=regularizers.l1(0.006),
                bias_regularizer=regularizers.l1(0.006) ,activation='leaky_relu')(x)

x = Dropout(rate=0.2)(x)       
output = Dense(class_count, activation='softmax')(x)
model4 = Model(inputs=base_model4.input, outputs=output)
lr = 0.001
model4.compile(Adamax(learning_rate=lr), loss='binary_crossentropy', metrics=['accuracy']) 

ValueError: ignored

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau

epochs = 150
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=0.00001)
callbacks=[reduce_lr]
history4 = model4.fit(x=train_gen,  epochs=epochs, verbose=1, callbacks=callbacks,  validation_data=valid_gen)

Epoch 1/150


KeyboardInterrupt: ignored